#### Analysis of EURO-CORDEX annual data

- The CARMINE Euro-Cordex selected experiments have an impact on the climate indicators for the CARMINE CSAs in Europe
- This notebook is designed to quantify the uncertainties of the indicators for the CSAs and to provide communication material for stakeholders and scientific conferences

In [1]:
from __future__ import annotations

from pathlib import Path
from typing import Iterable, List

def find_repo_root(start: Path) -> Path:
    for p in [start.resolve()] + list(start.resolve().parents):
        if (p / ".git").exists():
            return p
    raise RuntimeError("Could not find repo root (.git not found).")

REPO_ROOT = find_repo_root(Path.cwd())


# In this repository, the dataset tree lives directly under the repo root
DATA_ROOT: Path = REPO_ROOT

# Outputs (should NOT be committed)
OUT_ROOT: Path = REPO_ROOT / "outputs"
TABLE_DIR: Path = OUT_ROOT / "tables"
FIG_DIR: Path = OUT_ROOT / "figures"
TABLE_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR.mkdir(parents=True, exist_ok=True)

print("REPO_ROOT:", REPO_ROOT)
#print("DATA_ROOT:", DATA_ROOT)
print("TABLE_DIR exists:", TABLE_DIR.exists())
print("FIG_DIR exists:", FIG_DIR.exists())

REPO_ROOT: C:\Users\reinhvlr\OneDrive\CARMINE-T2.4
TABLE_DIR exists: True
FIG_DIR exists: True


In [2]:
# =============================================================================
# EURO-CORDEX test data paths (temporary testing workflows)
# =============================================================================

# Root folder containing zipped EURO-CORDEX test data
EUROCORDEX_TESTDATA_ROOT = (
    REPO_ROOT / "2601_EURO_CORDEX_testing_data"
    if "REPO_ROOT" in globals()
    else Path("./2601_EURO_CORDEX_testing_data")
)

# Unzip cache folder (keep it short to avoid Windows path length issues)
EUROCORDEX_TEST_UNZIP_ROOT = EUROCORDEX_TESTDATA_ROOT / "_unzipped"

# Dedicated output root for EURO-CORDEX test results
EUROCORDEX_TEST_OUTPUT_ROOT = (
    REPO_ROOT / "outputs" / "eurocordex_testing"
    if "REPO_ROOT" in globals()
    else Path("./outputs/eurocordex_testing")
)

# Create output directory (safe: ignored by git)
EUROCORDEX_TEST_OUTPUT_ROOT.mkdir(parents=True, exist_ok=True)

print("DATA:", EUROCORDEX_TESTDATA_ROOT)

DATA: C:\Users\reinhvlr\OneDrive\CARMINE-T2.4\2601_EURO_CORDEX_testing_data


In [3]:
# =============================================================================
# Unzip EURO-CORDEX test data (zip → _unzipped)
# =============================================================================
from pathlib import Path
import zipfile

zip_root = EUROCORDEX_TESTDATA_ROOT
unzip_root = EUROCORDEX_TEST_UNZIP_ROOT

zip_files = sorted(zip_root.rglob("*.zip"))
print(f"Found {len(zip_files)} zip file(s)")

for zpath in zip_files:
    out_dir = unzip_root / zpath.stem
    out_dir.mkdir(parents=True, exist_ok=True)

    print("\nZIP:", zpath.name)
    print(" -> unpack to:", out_dir)

    with zipfile.ZipFile(zpath, "r") as z:
        members = [m for m in z.namelist() if not m.endswith("/")]
        print("   entries:", len(members))

        written = []
        for m in members:
            target = out_dir / Path(m).name  # flatten: ignore internal paths
            with z.open(m) as src:
                target.write_bytes(src.read())
            written.append(target)

    print("   written files:", len(written))
    if written:
        print("   first file:", written[0].name, "bytes:", written[0].stat().st_size)

Found 2 zip file(s)

ZIP: 67a8bafca25cded2f165c94dc94b23ce.zip
 -> unpack to: C:\Users\reinhvlr\OneDrive\CARMINE-T2.4\2601_EURO_CORDEX_testing_data\_unzipped\67a8bafca25cded2f165c94dc94b23ce
   entries: 1
   written files: 1
   first file: tas_EUR-11_NCC-NorESM1-M_rcp26_r1i1p1_KNMI-RACMO22E_v1_mon_203101-204012.nc bytes: 41241048

ZIP: c7825f39aab277bfd373e79e60129936.zip
 -> unpack to: C:\Users\reinhvlr\OneDrive\CARMINE-T2.4\2601_EURO_CORDEX_testing_data\_unzipped\c7825f39aab277bfd373e79e60129936
   entries: 1
   written files: 1
   first file: tas_EUR-11_MPI-M-MPI-ESM-LR_rcp26_r1i1p1_SMHI-RCA4_v1a_mon_203101-204012.nc bytes: 38453239


In [8]:
p = r"C:\Users\reinhvlr\OneDrive\Documents\RUB\Projects\CARMINE\Work\carmine-analyses\data\CARMINE-T2.4VR\CARMINE-T2.4\2601_EURO_CORDEX_testing_data\_unzipped\67a8bafca25cded2f165c94dc94b23ce\tas_EUR-11_NCC-NorESM1-M_rcp26_r1i1p1_KNMI-RACMO22E_v1_mon_203101-204012.nc"
print("len(path) =", len(p))
print("len(dir)  =", len(p.rsplit("\\", 1)[0]))


len(path) = 261
len(dir)  = 185
